In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import random as rn
import os
import cne
import maxent

In [2]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
np.random.seed(SEED)
rn.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
triples = [('Eve', 'type', 'Lecturer'),
           #('Eve', 'type', 'Person'), 
           ('Lecturer', 'subClassOf', 'Person'), 
           #('David', 'type', 'Person'),
           ('David', 'type', 'Researcher'),
           ('Researcher', 'subClassOf', 'Person'),
           ('Flora', 'hasSpouse', 'Gaston'),
           ('Gaston', 'type', 'Person'),
           #('Flora', 'type', 'Person')
          ]

train = np.array(triples)

entities = np.unique(np.concatenate((train[:,0], train[:,2]), axis=0)).tolist()
relations = np.unique(train[:,1]).tolist()

num_entities = len(entities)
num_relations = len(relations)

ent2idx = dict(zip(entities, range(num_entities)))
rel2idx = dict(zip(relations, range(num_relations)))

idx2ent = {idx:ent for ent,idx in ent2idx.items()}
idx2rel = {idx:rel for rel,idx in rel2idx.items()}

In [4]:
#A = np.zeros(shape=(num_entities,num_entities,num_relations))
A = np.zeros(shape=(num_entities,num_entities))

for h,r,t in train:
    
    h_idx = entities.index(h)
    r_idx = relations.index(r)
    t_idx = entities.index(t)
    
    #A[h_idx,t_idx,r_idx] = 1
    A[h_idx, t_idx] = 1

def score(h,r,t):
    return tf.reduce_sum(tf.square(h + r - t))

In [5]:
train2idx = []

for head, rel, tail in train:
    
    head_idx = ent2idx[head]
    tail_idx = ent2idx[tail]
    rel_idx = rel2idx[rel]

    train2idx.append([head_idx, rel_idx, tail_idx])
    
train2idx = np.array(train2idx)

In [6]:
from scipy import sparse
A_sparse = sparse.csr_matrix(A)  

In [7]:
prior = maxent.BGDistr(A_sparse) 
prior.fit() 

In [8]:
CNE = cne.ConditionalNetworkEmbedding(
    A=A_sparse,
    d=10,
    s1=1,
    s2=1.5,
    prior_dist=prior
    )
CNE.fit(lr=.001, max_iter=100)

Epoch: 0, grad norm: 8.6828, obj: 162.7534, obj smoothness: 162.7534
Epoch: 1, grad norm: 8.6838, obj: 162.7317, obj smoothness: 0.0216
Epoch: 2, grad norm: 8.6848, obj: 162.7101, obj smoothness: 0.0217
Epoch: 3, grad norm: 8.6858, obj: 162.6884, obj smoothness: 0.0217
Epoch: 4, grad norm: 8.6868, obj: 162.6666, obj smoothness: 0.0218
Epoch: 5, grad norm: 8.6879, obj: 162.6448, obj smoothness: 0.0218
Epoch: 6, grad norm: 8.6889, obj: 162.6229, obj smoothness: 0.0219
Epoch: 7, grad norm: 8.6900, obj: 162.6009, obj smoothness: 0.0220
Epoch: 8, grad norm: 8.6911, obj: 162.5789, obj smoothness: 0.0220
Epoch: 9, grad norm: 8.6922, obj: 162.5568, obj smoothness: 0.0221
Epoch: 10, grad norm: 8.6933, obj: 162.5345, obj smoothness: 0.0222
Epoch: 11, grad norm: 8.6945, obj: 162.5122, obj smoothness: 0.0223
Epoch: 12, grad norm: 8.6956, obj: 162.4897, obj smoothness: 0.0225
Epoch: 13, grad norm: 8.6968, obj: 162.4671, obj smoothness: 0.0226
Epoch: 14, grad norm: 8.6980, obj: 162.4444, obj smoothn

In [9]:
X = CNE._ConditionalNetworkEmbedding__emb

In [10]:
from scipy.stats import halfnorm

def get_pij(i,j,s1,s2,prior, X):
    
    p_prior = prior.get_row_probability([i], [j])
    
    normal_s1 = halfnorm.rvs(loc=0,scale=s1,size=1)
    normal_s2 = halfnorm.rvs(loc=0,scale=s2,size=1)
    
    numerator = p_prior * normal_s1
    denom = numerator + (1-p_prior)*normal_s2
    
    return numerator/denom

In [ ]:
# x_i = np.array([1,2,3,4,5,6,7,8,9])
# x_j = np.array([0,1,0,0,0,0,0,0,0])
# np.outer(x_i, x_j)
#def get_hessian(i,j,k,P,A):
# P = np.array([[1,2,3],[4,5,6],[7,8,9]])
# A = np.array([[0,1,1],[1,0,0],[1,0,0]])
# print(P)
# print(A)
# print(P-A)

In [ ]:
#np.diag((P-A)[~np.eye(A.shape[0], dtype=bool)])
i=0
#identity=[]
s1 = 1
s2 = 1.5
gamma = (1/(s1**2)) - (1/(s2**2))

def get_hessian(i,s1,s2,gamma,X,A):
    
    hessian = np.zeros(shape=(10,10))

    for j in range(A.shape[0]):

        if i != j:

            x_i = X[i,:]
            x_j = X[j,:]

            x_diff = (x_i - x_j).reshape(-1,1)  
            
            prob = get_pij(i,j,1,1.5,prior, X)

            h = (gamma**2) * np.dot(x_diff,x_diff.T) * (prob * (1-prob))

            a = A[i,j]

            p_diff = gamma * (prob - a)[0]

            p_diff_mat = p_diff * np.identity(h.shape[0])

            hessian += p_diff_mat - h
            
    return hessian

In [ ]:
i=0
j=1
k=2

hessian = get_hessian(i,s1,s2,gamma,X,A)
pij = get_pij(i=i,j=j,s1=s1,s2=s2,prior=prior, X=X)

invert = (-hessian) / (gamma**2 * (pij) * (1-pij))

hess_inv = np.linalg.inv(-hessian)

x_i = X[i,:]
x_j = X[j,:]
x_k = X[k,:]

xij_diff = (x_i - x_j).reshape(1,-1)

xik_diff = (x_i - x_k)

np.dot(np.dot(xij_diff, hess_inv), xik_diff)

In [ ]:
#np.dot(np.dot(np.ones(shape=(3,1)), np.ones(shape=(1,3))), np.dot(P, 1-P))
# a = np.dot(P, 1-P)
# np.fill_diagonal(a, 0)
# a

In [ ]:
# with tf.GradientTape(persistent=True) as g:
    
#     A = tf.convert_to_tensor(A)

#     head = tf.argmax(A[:,tail_idx, rel_idx])
#     tail = tf.argmax(A[head_idx,:, rel_idx])
#     rel = tf.argmax(A[head_idx,tail_idx,:])
    
#     other_head = tf.argmax(A[:,tail_idx, rel_idx])
#     other_tail = tf.argmax(A[head_idx,:, rel_idx])
#     other_rel = tf.argmax(A[head_idx,tail_idx,:])
    
#     head_e,_,tail_e,_,rel_e= model([head,head,tail,tail,rel])

#     get_score = score(head_e,rel_e,tail_e)
    
# g.gradient(get_score,tail_e)